In [162]:
!pip install SBTi
!pip install datacompy

In [163]:
import pandas as pd

import SBTi
from SBTi.configs import ColumnsConfig
from SBTi.temperature_score import TemperatureScore, Scenario, ScenarioType, EngagementType
from SBTi.interfaces import ETimeFrames, EScope
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod

In [165]:
df_raw = pd.read_excel("data_provider_example.xlsx", sheet_name="sample")
df_raw

,company_name,company_id,isic,target_type,intensity_metric,ghg_s1s2,ghg_s3,scope,coverage_s1,coverage_s2,...,lar_time_range,time_frame,s1s2_reduction_ambition,s3_reduction_ambition,s1s2_lar,s3_lar,s1s2_ts,s3_ts,s3_emissions_percentage,ghg_s1s2s3_ts
0,Company A,10001,0,Absolute,0,10000,20000,S1+S2+S3,0.80,1,...,14,mid,0.560,0.350,0.0400,0.0250,1.447945,1.916439,0.666667,1.760274
1,Company B,10002,0,Absolute,0,5000,2500,S1+S2+S3,1.00,1,...,14,mid,0.700,0.700,0.0500,0.0500,1.135616,1.135616,0.333333,1.135616
2,Company C,10003,0,Absolute,0,1000000,750000,S1+S2+S3,1.00,1,...,14,mid,0.700,0.455,0.0500,0.0325,1.135616,1.682192,0.428571,1.369863
3,Company D,10004,0,Absolute,0,5000000,3000000,S1+S2+S3,0.65,1,...,14,mid,0.455,0.700,0.0325,0.0500,1.682192,1.135616,0.375000,1.682192
4,Company E,10005,0,Absolute,0,250000,50000,S1+S2+S3,0.66,1,...,14,mid,0.462,0.700,0.0330,0.0500,1.666575,1.135616,0.166667,1.666575
5,Company F,10006,0,Absolute,0,300000,500000,S1+S2+S3,1.00,1,...,14,mid,0.700,0.700,0.0500,0.0500,1.135616,1.135616,0.625000,1.135616
6,Company G,10007,0,Absolute,0,1000,500,S1+S2+S3,0.50,1,...,14,mid,0.350,0.700,0.0250,0.0500,1.916439,1.135616,0.333333,1.916439
7,Company H,10008,0,Absolute,0,75000,35000,S1+S2+S3,1.00,1,...,14,mid,0.700,0.700,0.0500,0.0500,1.135616,1.135616,0.318182,1.135616
8,Company I,10009,0,Absolute,0,10000000,4000000,S1+S2+S3,1.00,1,...,14,mid,0.700,0.700,0.0500,0.0500,1.135616,1.135616,0.285714,1.135616
9,Company J,10010,0,Absolute,0,4000,2000,S1+S2+S3,1.00,1,...,14,mid,0.700,0.700,0.0500,0.0500,1.135616,1.135616,0.333333,1.135616


In [182]:
provider = ExcelProvider("data_provider_example.xlsx")
portfolio = pd.read_excel("data_provider_example.xlsx", sheet_name="portfolio")
companies = SBTi.utils.dataframe_to_portfolio(portfolio)

time_frames = [ETimeFrames.MID]
scopes = [EScope.S1S2S3, EScope.S1S2, EScope.S3]
fallback_score = 1.5

temperature_score = TemperatureScore(
    time_frames=time_frames,
    scopes=scopes,
    fallback_score=fallback_score,
    aggregation_method=PortfolioAggregationMethod.ROTS 
)

df_scores = temperature_score.calculate(data_providers=[provider], portfolio=companies)
df_scores_subset = df_scores[["company_id", "scope", "reduction_ambition", "annual_reduction_rate", "temperature_score"]]

df_scores_subset_s1s2 = df_scores_subset[df_scores_subset["scope"] == EScope.S1S2].\
	rename(columns={"reduction_ambition": "s1s2_reduction_ambition", \
        "annual_reduction_rate": "s1s2_lar", \
        "temperature_score": "s1s2_ts" \
	}).drop(columns=["scope"])
df_scores_subset_s3 = df_scores_subset[df_scores_subset["scope"] == EScope.S3].\
	rename(columns={"reduction_ambition": "s3_reduction_ambition", \
        "annual_reduction_rate": "s3_lar", \
        "temperature_score": "s3_ts" \
	}).drop(columns=["scope"])
df_scores_subset_s1s2s3 = df_scores_subset[df_scores_subset["scope"] == EScope.S1S2S3]. \
	rename(columns={"temperature_score" : "ghg_s1s2s3_ts", \
	}).drop(columns=["reduction_ambition", "annual_reduction_rate", "scope"])

df_scores_merge = df_scores_subset_s1s2.merge(df_scores_subset_s3).merge(df_scores_subset_s1s2s3).astype({"company_id": "int64"}) 

df_scores_compare = df_scores_merge.merge(df_raw,left_on="company_id", right_on="company_id", suffixes=("_s", "_p"))

df_scores_compare[["company_id", "s1s2_lar_s", "s1s2_lar_p", "s1s2_ts_s", "s1s2_ts_p", "s3_lar_s", "s3_lar_p", "s3_ts_s", "s3_ts_p", "ghg_s1s2s3_ts_s", "ghg_s1s2s3_ts_p"]]

,company_id,s1s2_lar_s,s1s2_lar_p,s1s2_ts_s,s1s2_ts_p,s3_lar_s,s3_lar_p,s3_ts_s,s3_ts_p,ghg_s1s2s3_ts_s,ghg_s1s2s3_ts_p
0,10001,0.0400,0.0400,1.45,1.447945,0.0250,0.0250,1.92,1.916439,1.76,1.760274
1,10002,0.0500,0.0500,1.14,1.135616,0.0500,0.0500,1.14,1.135616,1.14,1.135616
2,10003,0.0500,0.0500,1.14,1.135616,0.0325,0.0325,1.68,1.682192,1.37,1.369863
3,10004,0.0325,0.0325,1.68,1.682192,0.0500,0.0500,1.14,1.135616,1.68,1.682192
4,10005,0.0330,0.0330,1.67,1.666575,0.0500,0.0500,1.14,1.135616,1.67,1.666575
5,10006,0.0500,0.0500,1.14,1.135616,0.0500,0.0500,1.14,1.135616,1.14,1.135616
6,10007,0.0250,0.0250,1.92,1.916439,0.0500,0.0500,1.14,1.135616,1.92,1.916439
7,10008,0.0500,0.0500,1.14,1.135616,0.0500,0.0500,1.14,1.135616,1.14,1.135616
8,10009,0.0500,0.0500,1.14,1.135616,0.0500,0.0500,1.14,1.135616,1.14,1.135616
9,10010,0.0500,0.0500,1.14,1.135616,0.0500,0.0500,1.14,1.135616,1.14,1.135616


In [183]:
aggregated_scores = temperature_score.aggregate_scores(df_scores)

In [151]:
def create_contributions_df(contributions):
	df = pd.DataFrame(
            columns = ["company_id", "company_name", "temperature_score", "contribution_relative", "contribution"])

	aggregated_scores.mid.S1S2.all.contributions

	for c in contributions:
		insert_loc = df.index.max()
		row = [c.company_id, c.company_name, c.temperature_score, c.contribution_relative, c.contribution] 

		if pd.isna(insert_loc):
			df.loc[0] = row
		else:
			df.loc[insert_loc + 1] = row
		
	return df

In [184]:
df_s1s2_contributions = create_contributions_df(aggregated_scores.mid.S1S2.all.contributions)
df_s3_contributions = create_contributions_df(aggregated_scores.mid.S3.all.contributions)
df_s1s2s3_contributions = create_contributions_df(aggregated_scores.mid.S1S2S3.all.contributions)

#df_s1s2_contributions.sort_values(by=["company_id"])	
df_s3_contributions.sort_values(by=["company_id"])	
#df_s1s2s3_contributions.sort_values(by=["company_id"])	


,company_id,company_name,temperature_score,contribution_relative,contribution
1,10001,Company A,1.92,5.620172,0.066333
5,10002,Company B,1.14,0.078406,0.000925
2,10003,Company C,1.68,3.512607,0.041458
0,10004,Company D,1.14,89.383312,1.054968
6,10005,Company E,1.14,0.039726,0.000469
4,10006,Company F,1.14,0.119178,0.001407
9,10007,Company G,1.14,0.002384,0.000028
7,10008,Company H,1.14,0.033370,0.000394
3,10009,Company I,1.14,1.191777,0.014066
8,10010,Company J,1.14,0.019068,0.000225


In [161]:
df_scores_merge

,company_id,s1s2_reduction_ambition,s1s2_lar,s1s2_ts,s3_reduction_ambition,s3_lar,s3_ts,ghg_s1s2s3_ts
0,10001,0.560,0.0400,1.45,0.350,0.0250,1.92,1.76
1,10002,0.700,0.0500,1.14,0.700,0.0500,1.14,1.14
2,10003,0.700,0.0500,1.14,0.455,0.0325,1.68,1.37
3,10004,0.455,0.0325,1.68,0.700,0.0500,1.14,1.68
4,10005,0.462,0.0330,1.67,0.700,0.0500,1.14,1.67
5,10006,0.700,0.0500,1.14,0.700,0.0500,1.14,1.14
6,10007,0.350,0.0250,1.92,0.700,0.0500,1.14,1.92
7,10008,0.700,0.0500,1.14,0.700,0.0500,1.14,1.14
8,10009,0.700,0.0500,1.14,0.700,0.0500,1.14,1.14
9,10010,0.700,0.0500,1.14,0.700,0.0500,1.14,1.14
